## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sisimiut,NaN,66.9395,-53.6735,25.54,49,0,7.05,clear sky
1,1,Garowe,NaN,8.4054,48.4845,76.64,56,6,17.78,clear sky
2,2,Ban Nahin,NaN,18.2418,104.2122,88.63,57,7,0.36,clear sky
3,3,Mangrol,NaN,21.1167,70.1167,81.55,81,94,5.53,overcast clouds
4,4,Mataura,NaN,-46.1927,168.8643,53.47,34,4,2.06,clear sky


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Garowe,NaN,8.4054,48.4845,76.64,56,6,17.78,clear sky
2,2,Ban Nahin,NaN,18.2418,104.2122,88.63,57,7,0.36,clear sky
3,3,Mangrol,NaN,21.1167,70.1167,81.55,81,94,5.53,overcast clouds
5,5,Avarua,NaN,-21.2078,-159.7750,78.85,69,20,3.44,few clouds
7,7,Yulara,NaN,-25.2406,130.9889,87.64,8,95,16.11,overcast clouds
8,8,Atuona,NaN,-9.8000,-139.0333,78.08,73,5,18.81,clear sky
9,9,San Policarpo,NaN,12.1791,125.5072,87.08,63,99,3.94,overcast clouds
10,10,Lamu,NaN,-2.2717,40.9020,77.22,80,9,14.12,clear sky
12,12,Porto Novo,NaN,6.4965,2.6036,76.82,90,99,2.80,overcast clouds
21,21,Payo,NaN,-3.7508,103.6372,80.33,84,91,1.70,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                696
City                   696
Country                232
Lat                    696
Lng                    696
Max Temp               696
Humidity               696
Cloudiness             696
Wind Speed             696
Current Description    696
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1421,Garowe,SO,76.64,clear sky,8.4054,48.4845,
1422,Ban Nahin,LA,88.63,clear sky,18.2418,104.2122,
1423,Mangrol,IN,81.55,overcast clouds,21.1167,70.1167,
1425,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
1427,Yulara,AU,87.64,overcast clouds,-25.2406,130.9889,
1428,Atuona,PF,78.08,clear sky,-9.8000,-139.0333,
1429,San Policarpo,PH,87.08,overcast clouds,12.1791,125.5072,
1430,Lamu,KE,77.22,clear sky,-2.2717,40.9020,
1432,Porto Novo,BJ,76.82,overcast clouds,6.4965,2.6036,
1441,Payo,ID,80.33,overcast clouds,-3.7508,103.6372,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1421,Garowe,SO,76.64,clear sky,8.4054,48.4845,Curubo Hotel
1422,Ban Nahin,LA,88.63,clear sky,18.2418,104.2122,
1423,Mangrol,IN,81.55,overcast clouds,21.1167,70.1167,Hotel The Royal Honours
1425,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
1427,Yulara,AU,87.64,overcast clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
...,...,...,...,...,...,...,...
2109,Trinidad,BO,84.81,few clouds,-14.8333,-64.9000,Hotel Piesta
2123,Mindelo,CV,78.35,broken clouds,16.8901,-24.9804,Hotel Gaudi
2124,Gibara,CU,79.92,clear sky,21.1097,-76.1317,Sol y Mar
2127,Dhidhdhoo,MV,79.83,light rain,6.8833,73.1000,Dhidhoo beach


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))